# Map-Matching

---

The map matching module is based on the python library `PyTrack`.

> ref: <https://github.com/cosbidev/PyTrack.git>

In [3]:
import numpy as np
import pandas as pd

# 1 Map-matching

`PyTrack` perfroms map-matchig based on a Hidden-Markov Model. Three steps are required for it:
1. Generate candidates for each point.
2. Extract a trellis DAG graph.
3. Perform viterbi search algorithm. 

> - A modification was made to avoid ``no candidates`` error in `trajtool.mapMatching`. To be specific, when no candidates are found for some points in the trajectory, the `candidate.elab_candidate_results` function will collapse due to `Key error`. In `trajtool`, we updated no-candidate results as an input to this function to avoid collapse.
> - Due to the data privacy, we can only provide processing procedure with limited samples.

In [4]:
# read traj
traj = pd.read_csv("./data/traj/sample_preprocessed.csv")
traj.head()

,vehID,orderID,tripID,time[s],lon,lat,dist[km],interval[s],speed[km/h],acc[m/s2],ele[m],grade[D],VSP[kW/t]
0,0.0,0.0,0.0,1.477969e+09,104.070087,30.731888,0.007380,1.0,32.384280,-3.726805,466.000,0.000000,-34.847050
1,0.0,0.0,0.0,1.477969e+09,104.070087,30.731822,0.004364,1.0,20.712196,-1.527181,466.000,0.068639,-9.830153
2,0.0,0.0,0.0,1.477969e+09,104.070082,30.731783,0.003779,1.0,15.737670,0.747944,465.700,0.055508,3.349933
3,0.0,0.0,0.0,1.477969e+09,104.070070,30.731750,0.005502,1.0,18.587265,2.996978,465.490,0.026709,17.738155
4,0.0,0.0,0.0,1.477969e+09,104.070051,30.731704,0.009463,1.0,29.423483,5.278276,465.343,0.042550,50.495445


`PyTrack` extracts road graph from osm directly according to a bounding box of the input coordinates. Extractions are performed using `requests` module.  

Please ensure the connectivity of the Internet.

In [5]:
from mapmatch import Matcher

matcher = Matcher(engine='pytrack')
traj = matcher.match(traj)
traj.head()

Graph extracting...
Downloaded 1,876.50kB


Map-matching: 100%|██████████| 11/11 [00:08<00:00,  1.35it/s]


,vehID,orderID,tripID,time[s],lon,lat,dist[km],interval[s],speed[km/h],acc[m/s2],ele[m],grade[D],VSP[kW/t],osmid
0,0.0,0.0,0.0,1.477969e+09,104.070087,30.731888,0.007380,1.0,32.384280,-3.726805,466.000,0.000000,-34.847050,None
1,0.0,0.0,0.0,1.477969e+09,104.070087,30.731822,0.004364,1.0,20.712196,-1.527181,466.000,0.068639,-9.830153,None
2,0.0,0.0,0.0,1.477969e+09,104.070082,30.731783,0.003779,1.0,15.737670,0.747944,465.700,0.055508,3.349933,None
3,0.0,0.0,0.0,1.477969e+09,104.070070,30.731750,0.005502,1.0,18.587265,2.996978,465.490,0.026709,17.738155,None
4,0.0,0.0,0.0,1.477969e+09,104.070051,30.731704,0.009463,1.0,29.423483,5.278276,465.343,0.042550,50.495445,None


In [6]:
traj[~traj['osmid'].isna()]

,vehID,orderID,tripID,time[s],lon,lat,dist[km],interval[s],speed[km/h],acc[m/s2],ele[m],grade[D],VSP[kW/t],osmid
510,0.0,0.0,2.0,1.477970e+09,104.094809,30.711877,0.007716,1.0,35.405457,-1.755613,464.000151,0.000006,-29.716919,137130750
511,0.0,0.0,2.0,1.477970e+09,104.094885,30.711854,0.007485,1.0,29.483807,0.154507,464.000106,0.000004,-3.119469,137130750
512,0.0,0.0,2.0,1.477970e+09,104.094962,30.711840,0.008458,1.0,30.159600,0.805633,464.000074,0.000003,6.311605,137130750
513,0.0,0.0,2.0,1.477970e+09,104.095040,30.711805,0.009543,1.0,33.095745,0.505840,464.000052,0.000002,5.397991,137129174
514,0.0,0.0,2.0,1.477970e+09,104.095115,30.711749,0.009920,1.0,34.927532,-0.060676,464.000036,0.000001,0.555017,137129174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2651,0.0,4.0,11.0,1.477983e+09,104.043225,30.704992,0.011768,1.0,42.088410,-1.356198,463.077291,0.078286,-6.348315,577379395
2652,0.0,4.0,11.0,1.477983e+09,104.043123,30.704933,0.009754,1.0,37.206097,-2.660773,462.154104,0.096709,-16.411404,577379395
2653,0.0,4.0,11.0,1.477983e+09,104.043039,30.704884,0.006534,1.0,27.627314,-1.874143,461.207873,0.055402,-8.889084,577379395
2654,0.0,4.0,11.0,1.477983e+09,104.042983,30.704850,0.004997,1.0,20.880402,0.044494,460.845511,0.050719,2.520604,100305570


In [7]:
# traj.to_csv("./data/traj/sample_mapmatched.csv", index=False)